# Newton Raphson
This short notebook illustrate how to offlaod part of the calculation of a Newton Raphson algorithm to a quantum computers. Several options are possible that we are exploring here.

The Newton Raphson algorithm allows to solve a set of non-linear equations:

$$
f_1(x_1,x_2, ..., x_n) = 0 \\
f_2(x_1,x_2, ..., x_n) = 0 \\
... \\
f_n(x_1,x_2, ..., x_n) = 0 \\
$$

where $x_i$ is a given variable and $f_i$ a general non-linear function.  This is generally written as 

$$
F(X) := \begin{pmatrix} f_1(X) \\ f_2(X) \\ ... \\ f_n(X) \end{pmatrix} = 0
$$ 

Assuming the functions $f_j$ are differentiable we define the Jacobian of $F$ at a point $X_k$ as :

$$
J(X_k) = \begin{pmatrix}
\partial_1 f_1(X_k) &  \partial_2 f_1(X_k) & ... & \partial_n f_1(X_k) \\
\partial_1 f_2(X_k) &  \partial_2 f_2(X_k) & ... & \partial_n f_2(X_k) \\
... & ... & ... & ... \\
\partial_1 f_n(X_k) &  \partial_2 f_n(X_k) & ... & \partial_n f_n(X_k) \\
\end{pmatrix}
$$


Given a point $X_k$ we can find a point closer to the solution by linearizing $F(X)=0$ around $X_k$ and solving the linear equation. In other words we find the next point, $X_{k+1}$ by solving the linear equation:

$$
F(X_k) + J(X_k)(X - X_k) = 0
$$

In pseudo algorithm this us expressed as:

1. Start with an initial guess $X_0$ and set $k=0$
2. while$ $|F(X_k)| > \epsilon$ :
    * Compute the soluion $H_k$ of the linear equation : $J(X_k) H_k = - F(X_k)$
    * Update the solution $X_{k+1} = X_{k} + H_{k}$

## Example 
To make things clearer let's consider the following example:

$$
f_1(x,y,z) = 2x^3 - x y+ 4z -12 \\
f_2(x,y,z) = -4x + z^5 + 8 z y \\
f_3(x,y,z) = 3x^3 +z^5 + 8 z y
$$

That we define in the following function

In [ ]:
import numpy as np
def func(input):
    def f1(x,y,z):
        return 2*x**3 - x*y + 4*z - 12
    def f2(x,y,z):
        return -4*x + z**5 + 8*z*y
    def f3(x,y,z):
        return 3*x**3 + z**5 + 8*z*y
    x, y, z = input
    return np.array([f(x,y,z) for f in[f1,f2,f3]])

As seen above we will also need the Jacobian of this function. Given the simplicity of the equation we can have the analytical derivatives  

In [ ]:
def grad(input):
    def df1(x,y,z):
        return np.array([6*x**2 - y, -x, 4]) 
    def df2(x,y,z):
        return np.array([-4, 8*z, 5*z**4 + 8*y])
    def df3(x,y,z):
        return np.array([9*x**2, 8*z, 5*z**4 + 8*y])
    out = np.zeros((3,3))
    x,y,z = input
    out[0,:] = df1(x,y,z)
    out[1,:] = df2(x,y,z)
    out[2,:] = df3(x,y,z)

    return out

## LU Decomposition 

The most straightforward way to solve the linear system of equations is to compute the LU decomposition of the matrix of the linear system (i.e. $J(X_k)$). Given a LU decomposition $A = LU$ where $L$ and $U$ are Lower and Upper tridiagonal matrices, one can write:

$$
A x = b \\
L U  x = b \\
L y = b  \\
U x = y 
$$

The equation $Ly=b$ is solveed using forward substitution and $U X = y$ with backward substitution. This approach therefore replace the linear system $Ax=b$ with two easy to solve linear systems $Ly=b$ and $Ux=y$. 

One issue appears when the matrix $A$ is very large but very sparse. The LU decomposition can results in a drastic loss of sparsity leading to $L$ and $U$ matrices that are dense. This can lead to memory issues where not enough space is available to store these matrices. 

Rearranging the elements of the matrix can lead to a significant improvement of the sparsity of $L$ and $U$, but finding an optimal reordering is difficult. Some classical heuristic approaches have been developped are are already implemented in the splu solver



In [ ]:
from quantum_newton_raphson.newton_raphson import newton_raphson
initial_point = np.random.rand(3)
res = newton_raphson(func, initial_point, grad=grad)
assert np.allclose(func(res.solution), 0)

The gradients of the function can also be computed numerically through finite difference. This is what happens if the argument `grad` is no specified 

In [ ]:
res = newton_raphson(func, initial_point)
assert np.allclose(func(res.solution), 0)